# Classes for the O2 solver: voxelized method

### Grid class

In [20]:
import numpy as np, matplotlib.pyplot as plt, numba
from typing import Union, Tuple, List
from itertools import groupby
import graph_tool as gt
from graph_tool.generation import lattice
from graph_tool.draw import graph_draw
from tqdm.notebook import tqdm
import cProfile as profile
from numpy import ma
import cProfile as profile
from numpy import ma
from matplotlib.collections import LineCollection

### Some utils

In [21]:
def get_prop_type(value, key=None):
    """
    Performs typing and value conversion for the graph_tool PropertyMap class.
    If a key is provided, it also ensures the key is in a format that can be
    used with the PropertyMap. Returns a tuple, (type name, value, key)
    """
    
    if key=='position':
        tname = 'vector<float>' # Somehow using vectors doesn't allow for substraction
    elif key == 'plexus':
        tname = 'short'
        value = int(value)
    elif key == 'stage':
        tname = 'short'
        try: 
            value = int(value)
        except ValueError:
            value = 10
    elif key=='nodeType':
        tname = 'string'
        value = str(value)
    elif key in ['radius','pressure','length','hd','flow']:
        tname = 'float'
        try:
            value = float(value)
        except ValueError:
            value = 0.0

    else:    

        if isinstance(key, str):
            # Encode the key as ASCII
            tname = 'string'
            key = key.encode('ascii', errors='replace')

        # Deal with the value
        if isinstance(value, bool):
            tname = 'bool'

        elif isinstance(value, int):
            tname = 'float'
            value = float(value)

        elif isinstance(value, float):
            tname = 'float'

        elif isinstance(value, str):
            tname = 'string'
            value = str(value.encode('ascii', errors='replace'))

        elif isinstance(value, dict):
            tname = 'object'

        else:
            tname = 'string'
            value = str(value)

    return str(tname), value, key

@numba.njit
def DecompressRLE(RLElabels:np.ndarray, arrayShape) -> np.ndarray:
    '''
    Decompress the grid labels into a full array of int.
    '''
    res = np.empty(arrayShape[0]*arrayShape[1]*arrayShape[2], np.uint8)
    ii = 0
    for i in range(0, RLElabels.size, 2):
        value = RLElabels[i]
        rep   = RLElabels[i+1]
        for j in range(0,rep):
            res[ii] = value
            ii += 1
    res = res.reshape(arrayShape)
    return res
    
def CompressRLE(labels:np.ndarray) -> np.ndarray:
    '''Compress the grid labels to RLE.'''
    return np.asarray([[val, sum(1 for i in group)] for val,group in groupby(labels.flat)]).flatten()

### Previous version, somehow voxels are translated.

class Grid(object):
    def __init__(self, origin:tuple[float], sideLengths:tuple[float], shape:tuple[int])->None:
        """
        Initialize a 3D grid with the specified shape.

        Parameters:
        - origin (tuple): The location of the bottom, left, front corner (i.e., index 0 o the grid), e.g., (0,0,0)
        - sideLengths (tuple): The length of each side of the grid (e.g., (1.0,1.2,1.5))
        - shape (tuple): The shape of the grid (e.g., (3, 3, 3)).
        """
        self.n = np.asarray(shape)
        self.origin = np.asarray(origin)
        self.l = np.asarray(sideLengths)
        self.h = np.asarray([l/n for l,n in zip(self.l, self.n)])
        self.labels = {}

    def __getitem__(self, idx)->float:
        return self.grid[idx]
    def __iter__(self):
        for i in range(self.size):
            try:
                yield self.labels[i]
            except KeyError:
                yield 0

    @property
    def shape(self)->tuple[int]:
        return self.n
    @property
    def size(self)->int:
        return np.prod(self.n)

    def GetItem(self, flatIndex:int)->float:
        '''Returns grid entry based on a flat index.'''
        return self.flatGrid[flatIndex]    

    def GetNeighbors(self, i,j,k)->np.ndarray:
        """Get the direct neighbors of a cell at the specified indices."""
        neighbors = []
        for di in range(-1, 2):
            for dj in range(-1, 2):
                for dk in range(-1, 2):
                    if not 0<sum(abs(d) for d in (di,dj,dk))<=1:
                        continue # Skip diagonal neighbours and call to cell itself
                    ni, nj, nk = i + di, j + dj, k + dk
                    if 0 <= ni < self.shape[0] and 0 <= nj < self.shape[1] and 0 <= nk < self.shape[2]:
                        neighbors.append(self.grid[ni,nj,nk])
        return np.asarray(neighbors)
    
    def ToFlatIndex(self, i:int,j:int,k:int)->int:
        '''Returns the index in the flattened array.'''
#         return self.shape[1] * (k*self.shape[2] + j) + i
        return np.ravel_multi_index((i,j,k), dims=self.shape, mode='wrap')

    def To3DIndex(self, idx:int)->tuple[int]:
        '''Returns the 3D index.'''
#         k = idx // (self.shape[2]*self.shape[1])
#         j = (idx - k*self.shape[2]*self.shape[1]) // self.shape[2]
#         i = idx - self.shape[1] * (j + self.shape[2]*k)
#         return (i,j,k)
        return np.unravel_index(idx, shape=self.shape)


    def ProbeAlongLine(self, start:tuple[float], end:tuple[float]):
        '''
        TODO, see https://code.activestate.com/recipes/578112-bresenhams-line-algorithm-in-n-dimensions/
        '''
        print("Not implemented yet.")
        return
    
    def Dist(self,
             cell1 : Union[List[int], Tuple[int], np.ndarray],
             cell2 : Union[List[int], Tuple[int], np.ndarray]) -> int:
        return int(np.sum(np.abs(np.array(cell1)-np.array(cell2))))

    def _BoundingBoxOfVessel(self, p1 : np.ndarray, p2 : np.ndarray,
                             r : float) -> Tuple[np.ndarray, np.ndarray]:
        '''
        Finds the bounding box for the vessel with end points p1 and p2
        in cell coordinates (i,j,k indices).
        For formulas, see https://iquilezles.org/articles/diskbbox/
        '''
        n = (p1-p2) # The axis of the cylinder
        n = r*(1-(n/np.linalg.norm(n))**2)**0.5 # The direction, orthogonal to the axis,
                                                # where to pick the bounding box corners
        # Find bounding box for the cylinder as the bounding box of the
        # bounding boxes of its caps (the disk faces)
        bboxes = np.array([p1 - n, p1 + n, p2 - n, p2 + n])
        cellMin, cellMax = self.PointToCell(bboxes.min(axis=0)), self.PointToCell(bboxes.max(axis=0))
        return np.maximum(cellMin, [0,0,0]), np.minimum(cellMax, self.n-1)
    
    def PointToCell(self, X: Union[List[float], Tuple[float], np.ndarray]) -> np.ndarray:
        # Center and normalize
        X = np.divide(np.asarray(X).reshape((3,-1)) - self.origin[:,np.newaxis], self.l[:,np.newaxis]) 
        xVoxelSpace = np.multiply(X, self.n[:,np.newaxis]) # Project onto voxel space
        return np.floor(xVoxelSpace).astype(int)
    
    def CellCenter(self, ijk : Union[np.ndarray, List[int], int, Tuple[int]]) -> np.ndarray:
        if isinstance(ijk, int):
            ijkarr = np.asarray(self.To3DIndex(ijk))
        ijkarr = np.asarray(ijk).reshape((3,-1)) 
        if ((ijkarr-self.n[:,np.newaxis]) > 0).any():
            raise ValueError(f"Indices {ijkarr[((ijkarr-self.n[:,np.newaxis])>0).any(1)]} out of bounds for the grid.")        
        cellCenter = self.origin[:,np.newaxis] + self.l[:,np.newaxis] * (ijkarr/self.n[:,np.newaxis])
        return np.squeeze(cellCenter)
    
    # def CompressLabels(self) -> None:
    #     '''Compress the labels from array to RLE format once no longer needed.'''
    #     self.grid = DecompressRLE(self.grid)
    # def DecompressLabels(self)->None:
    #     '''Recover the labels array from RLE format.'''
    #     self.grid = DecompressRLE(self.grid, self.n)
class VascularGraph(gt.Graph):
    def __init__(self, graphFile:str=None) -> None:
        super(VascularGraph, self).__init__(directed=True)
        if graphFile:
            self.graphFile = graphFile
            self._ReadFromFile(self.graphFile)

    def _ReadFromFile(self, graphFile:str):
        def ifFloat(x):
            try:
                return float(x)
            except ValueError:
                return x
        
        self.clear()
        with open(graphFile, 'r') as f:
            line = f.readline()
            while not line.startswith('# Nodes'):
                line = f.readline()
            nNodes = int(line.split(' ')[-1])
            attributes = f.readline().strip('\n').split(',')[1:] # First item is node name
            nodes = []
            for n in range(nNodes):
                node = f.readline().strip('\n').split(',')
                nodes.append((node[0], [ifFloat(node[i+1]) if attributes[i]!='position'
                                        else np.array(list(map(float, node[i+1].strip('[').strip(']').split()))) 
                                        for i in range(len(attributes))]))
            nodeAttributes = [n for n in attributes] # Copy
            line = f.readline()
            while not line.startswith('# Edges'):
                line = f.readline()
            nEdges = int(line.split(' ')[-1])
            attributes = f.readline().strip('\n').split(',')[2:] # First two items are node names
            edges = []
            for n in range(nEdges):
                edge = f.readline().split(',')
                edges.append((edge[0], edge[1], [ifFloat(edge[i+2]) for i in range(len(attributes))]))
            edgeAttributes = [n for n in attributes] # Copy
            
        # Convert to integers before adding to graph
        nodesToInt = {n[0]:i for i,n in enumerate(nodes)}
        nodes = [[nodesToInt[n], [get_prop_type(value, key)[1] for key, value in zip(nodeAttributes, d)]] for n,d in nodes]    
        edges = [(nodesToInt[n1], nodesToInt[n2], *[get_prop_type(value, key)[1] for key, value in zip(edgeAttributes, d)]) for n1,n2,d in edges]
        self.add_edge_list(edges,
                    eprops=[get_prop_type(value, key)[::-2] for key, value in zip(edgeAttributes, edges[0][2:])],        
                    )
        del edges, nodesToInt
        
        vprops = [self.new_vertex_property(get_prop_type(value,key)[0]) for key, value in zip(nodeAttributes, nodes[0][1])]
        for i, vprop in enumerate(vprops):
            self.vp[nodeAttributes[i]] = vprop
            if vprop.value_type() not in ['python::object','string', 'vector<double>', 'vector<long double>']:
                vprop.a = [d[i] for _,d in nodes]
            elif vprop.value_type() in ['string', 'python::object']:
                for n,d in nodes:
                    vprop[n] = d[i]
            else:
                vprop.set_2d_array(np.array([d[i] for _,d in nodes]).T, pos=np.array([n for n,_ in nodes]))
        del nodes
        return 
    
    def SplitVesselsToMeshSize(self, h:float, vessels:list):
        pass

# VAG = VascularGraph()    
class ControlVolumes(object):
        '''
        Creates the control volumes within the Grid and stores the connections and their types
        between the grid and the control volumes (endothelial and (optional) vascular voxels)
        and between the vascular nodes and the control volumes.
        '''
        def __init__(self, grid:Grid, vessels:VascularGraph):
            # Compute distance to grid.origin along each axis
            d = vessels.new_vertex_property("bool")
            pos = vessels.vp['position']
            # #inFOV = (np.abs(np.array([pos[u] for u in vessels.iter_vertices()])-grid.origin)<=grid.l)
            # positions = vessels.vp['position'].get_2d_array([0,1,2]) - grid.origin.reshape((3,1))
            
            points = vessels.vp['position'].get_2d_array([0,1,2]) 
            d.a = np.logical_and((points<=grid.origin[:,np.newaxis]+grid.l[:,np.newaxis]), (points>=grid.origin[:,np.newaxis])).all(0)
            self.grid = grid
            self._vessels = vessels # Link to the original graph, if needed
            self.VAG = gt.GraphView(vessels, vfilt=d)

        def _RotationMatrix(self, n:np.ndarray):
            """Rotate a capsule onto the vessel axis n."""
            if (n==[0,0,1]).all():
                return np.eye(3)
            if (n==[0,0,-1]).all():
                R = np.array([[0,0,0],[0,0,0],[0,0,-1]])                                
            R = (n[2]-1)/(n[0]**2+n[1]**2) * np.array([[n[0]**2, n[0]*n[1], 0],
                                                        [n[0]*n[1], n[1]**2, 0],
                                                        [0, 0, n[0]**2+n[1]**2]])                      
            R+= np.array([[1, 0, -n[0]],
                            [0, 1, -n[1]],
                            [n[0], n[1], 1]])                              
            return R
        
        def _LabelRefCapsule(self, r:float, l:float, w:float,):
                            # nx:int=50, ny:int=50, nz:int=100):
            """
            Samples point around the refrence capsule 
            (a vertical cylinder of length l and radius r)
            and labels them according to the lumen radius and vessel wall thickness (w).
            """    
            # Creates a 3D grid, large enough for the capsule to fit in
            # pos    = np.meshgrid(np.linspace(-1.1*(r-w), 1.1*(r+w), endpoint=True, num=nx),
            #                     np.linspace(-1.1*(r-w), 1.1*(r+w), endpoint=True, num=ny),
            #                     np.linspace(0, l, endpoint=True, num=nz)) 
            width = r+2*w # Width of the cylinder
            pos = np.meshgrid(np.arange(-width, width, w/5),
                              np.arange(-width, width, w/5),
                              np.arange(0, l, l/30))
            nx,ny,nz = pos[0].shape
            pos = np.vstack(list(map(lambda x:x.ravel(), pos))) # (3,nx*ny*nz) array of locations
            dists = np.linalg.norm(pos[:2], axis=0) # Distance from the cylinder axis
            labels = np.where(dists<r, 1, np.where(dists<r+w, 2, 0))
            labels[dists<r] = 1
            return pos, labels   
        
        def _LabelAroundVessel(self, e:list, w:float=1e-4):
            ''' 
            TODO: look for the closest node in the edge to assign the connectivity to. Should then return 4 sets (2 for each node). 
            ### Parameters:
            - e: list
                An edge.
            - w: float
                The endothelium thickness (in cm).
            '''

            p0, p1 = [self.VAG.vp['position'][n].a for n in e]
            r, l   = self.VAG.ep['radius'][e], self.VAG.ep['length'][e] 
            R = self._RotationMatrix((p1-p0)/l) # Rotation operator to the vessel axis

            pos, labels = self._LabelRefCapsule(r, l, 0.001) # Return a grid with its labels on a vertical capsule
            pos = p0[:, np.newaxis] + l*R.T.dot(pos) # Transpose and translate the capsule to the real vessel's axis
            gridDomain = self.grid.PointToCell(pos) # Maps the points in the reference capsule to voxels in our grid
            mask = (gridDomain<self.grid.n[:,np.newaxis]).all(0) # Some points may be outside the grid after rotation, so we mask them
            ijk = self.grid.ToFlatIndex(*gridDomain)[mask] # Maps the ijk indices to flat array indices
            try:
                endothelium = ijk[labels[mask]==2]
                vascular    = ijk[labels[mask]==1]
            except IndexError:
                print(ijk.shape, labels.shape, labels[mask].shape)
                raise IndexError("")
            vascular = set(vascular.flat) # The vascular labels
            endothelium = set(endothelium) # The endothelium labels
            vascular = vascular-endothelium # Endothelium label is preffered over vascular  
            del pos, ijk, labels, gridDomain # Clean up          
            return endothelium, vascular
            
        def LabelMesh(self, w:float=1e-4)->None:
            '''
            TODO: see _LabelAroundVessel.
            Labels the mesh and saves the connectivity between the control volumes, the mesh and the vascular nodes. 
            ### Parameters
            - w: float
                The endothelium thickness (in cm).
            '''

            endo = self.VAG.new_vertex_property("python::object", val=())
            vasc = self.VAG.new_vertex_property("python::object", val=())
            for e in tqdm(self.VAG.iter_edges(), 
                          total=self.VAG.num_edges(), 
                          desc="Labelling...", unit="edge"): # Loop through the vessels in the FOV.
                # This should loop through the edges within the FOV, but the index of the vertices
                # is the same as in the original graph (therefore, the indices do not necesseraly start at 0).
                endo[e[0]], vasc[e[0]] = self._LabelAroundVessel(e, w) # Save the sets of connections to the grid 
                                                                       # as vertex properties of the proximal node.
                                                                       # Should be connected to the closest node, but that's simpler.
            self.VAG.vp['endothelium'] = endo
            self.VAG.vp['vascular']    = vasc

        def bar(self): # To profile the time for labelling.
            profile.runctx('self.LabelMesh()', globals(), locals())

        def LabelsToGrid(self):
            # Write the labels to the grid for plotting.
            try:
                self.grid.labels = {i:1 for idx in self.VAG.vp['vascular'] for i in idx}
                self.grid.labels = {**self.grid.labels, **{i:2 for idx in self.VAG.vp['endothelium'] for i in idx}}        
            except KeyError:
                self.grid.labels = {} # No labels.
            
        def ToVTK(self, VTKFileName:str="gridLabels.vtk"): 
            self.LabelsToGrid()
            with open(VTKFileName, 'w') as f:
                f.write("# vtk DataFile Version 5.1\n")
                f.write("A mesh for the computation of oxygen perfusion.\n")
                #f.write("BINARY\n")
                f.write("ASCII\n")
                f.write("DATASET STRUCTURED_POINTS\n")
                f.write("DIMENSIONS {:d} {:d} {:d}\n".format(*(self.grid.n+1)))
                f.write("ORIGIN {:f} {:f} {:f}\n".format(*self.grid.origin))
                f.write("SPACING {:f} {:f} {:f}\n".format(*self.grid.h))
            
                # Writing the data
                f.write(f"CELL_DATA {self.grid.size}\n")
                f.write("SCALARS labels int 1\n")
                f.write("LOOKUP_TABLE default\n")
                
                a = np.array([self.grid.labels.get(i,0) for i in 
                              tqdm(np.ravel_multi_index(
                                  self.grid.To3DIndex(np.arange(self.grid.size)), 
                                  self.grid.shape, order='F'), unit='labels')])
                a.tofile(f, sep='\n', format='%d')
                del a
#                 f.write(('\n'.join(i.__str__() for i in 
#                                    tqdm(self.grid, desc=f"Writing labels to {VTKFileName}", 
#                                         total=self.grid.size, unit=' Labels'))))  
            return          

        def _PlotPlexus(self, plexus:int, output=None):
            _G = gt.GraphView(self.VAG, vfilt=self.VAG.vp['plexus'].fa==plexus)
            if _G.num_vertices()==0:
                 print(f"No vertices in the " + ("SVP" if plexus==0 else
                                             "ICP" if plexus==1 else
                                             "DCP") + " layer. Not plotting.")
                 return
            graph_draw(_G, pos=self.VAG.vp['position'], bg_color='white', output=output)
            del _G
        def PlotSVP(self, output=None):
            self._PlotPlexus(plexus=0, output=output)             
        def PlotICP(self, output=None):
            self._PlotPlexus(plexus=1, output=output)
        def PlotDCP(self, output=None):
            self._PlotPlexus(plexus=2, output=output)           

# CV = ControlVolumes(Grid((0,0,0), (.1,0.1,.3), (100,100,100)), VAG)
# CV.PlotICP()

### Working version

In [35]:
class Grid(object):
    def __init__(self, origin:tuple[float], sideLengths:tuple[float], shape:tuple[int])->None:
        """
        Initialize a 3D grid with the specified shape.

        Parameters:
        - origin (tuple): The location of the bottom, left, front corner (i.e., index 0 o the grid), e.g., (0,0,0)
        - sideLengths (tuple): The length of each side of the grid (e.g., (1.0,1.2,1.5))
        - shape (tuple): The shape of the grid (e.g., (3, 3, 3)).
        """
        self.n = np.asarray(shape)
        self.origin = np.asarray(origin)
        self.l = np.asarray(sideLengths)
        self.h = np.asarray([l/(n-1) for l,n in zip(self.l, self.n)])
        self.labels = {}

    def __getitem__(self, idx)->float:
        return self.grid[idx]
    def __iter__(self):
        for i in range(self.size):
            try:
                yield self.labels[i]
            except KeyError:
                yield 0

    @property
    def shape(self)->tuple[int]:
        return self.n
    @property
    def size(self)->int:
        return np.prod(self.shape)

    def GetItem(self, flatIndex:int)->float:
        '''Returns grid entry based on a flat index.'''
        return self.flatGrid[flatIndex]    

    def GetNeighbors(self, i,j,k)->np.ndarray:
        """Get the direct neighbors of a cell at the specified indices."""
        neighbors = []
        for di in range(-1, 2):
            for dj in range(-1, 2):
                for dk in range(-1, 2):
                    if not 0<sum(abs(d) for d in (di,dj,dk))<=1:
                        continue # Skip diagonal neighbours and call to cell itself
                    ni, nj, nk = i + di, j + dj, k + dk
                    if 0 <= ni < self.shape[0] and 0 <= nj < self.shape[1] and 0 <= nk < self.shape[2]:
                        neighbors.append(self.grid[ni,nj,nk])
        return np.asarray(neighbors)
    
    def ToFlatIndex(self, i:int,j:int,k:int)->int:
        '''Returns the index in the flattened array.'''
        #return self.shape[1] * (i*self.shape[0] + j) + k
        return np.ravel_multi_index((i,j,k), dims=self.shape, mode='wrap')
    def To3DIndex(self, idx:int)->tuple[int]:
        '''Returns the 3D index.'''
        # i = idx // (self.shape[0]*self.shape[1])
        # j = (idx - i*self.shape[0]*self.shape[1]) // self.shape[0]
        # k = idx - self.shape[1] * (j + self.shape[0]*i)
        # return (i,j,k)
        return np.unravel_index(idx, shape=self.shape)


    def ProbeAlongLine(self, start:tuple[float], end:tuple[float]):
        '''
        TODO, see https://code.activestate.com/recipes/578112-bresenhams-line-algorithm-in-n-dimensions/
        '''
        print("Not implemented yet.")
        return
    
    def Dist(self,
             cell1 : Union[List[int], Tuple[int], np.ndarray],
             cell2 : Union[List[int], Tuple[int], np.ndarray]) -> int:
        return int(np.sum(np.abs(np.array(cell1)-np.array(cell2))))

    def _BoundingBoxOfVessel(self, p1 : np.ndarray, p2 : np.ndarray,
                             r : float) -> Tuple[np.ndarray, np.ndarray]:
        '''
        Finds the bounding box for the vessel with end points p1 and p2
        in cell coordinates (i,j,k indices).
        For formulas, see https://iquilezles.org/articles/diskbbox/
        '''
        n = (p1-p2) # The axis of the cylinder
        n = r*(1-(n/np.linalg.norm(n))**2)**0.5 # The direction, orthogonal to the axis,
                                                # where to pick the bounding box corners
        # Find bounding box for the cylinder as the bounding box of the
        # bounding boxes of its caps (the disk faces)
        bboxes = np.array([p1 - n, p1 + n, p2 - n, p2 + n])
        cellMin, cellMax = self.PointToCell(bboxes.min(axis=0)), self.PointToCell(bboxes.max(axis=0))
        return np.maximum(cellMin, [0,0,0]), np.minimum(cellMax, self.n-1)
    
    def PointToCell(self, X: Union[List[float], Tuple[float], np.ndarray]) -> np.ndarray:
        # Center and normalize
        X = np.divide(np.asarray(X).reshape((3,-1)) - self.origin[:,np.newaxis], self.l[:,np.newaxis]) 
        xVoxelSpace = np.multiply(X, self.n[:,np.newaxis]) # Project onto voxel space
        return np.floor(xVoxelSpace).astype(int)
    
    def CellCenter(self, ijk : Union[np.ndarray, List[int], int, Tuple[int]]) -> np.ndarray:
        if isinstance(ijk, int):
            ijkarr = np.asarray(self.To3DIndex(ijk))
        ijkarr = np.asarray(ijk).reshape((3,-1)) 
        if ((ijkarr-self.n[:,np.newaxis]) > 0).any():
            raise ValueError(f"Indices {ijkarr[((ijkarr-self.n[:,np.newaxis])>0).any(1)]} out of bounds for the grid.")        
        cellCenter = self.origin[:,np.newaxis] + self.l[:,np.newaxis] * ((0.5+ijkarr)/self.n[:,np.newaxis])
        return np.squeeze(cellCenter)
    
class VascularGraph(gt.Graph):
    def __init__(self, graphFile:str=None) -> None:
        super(VascularGraph, self).__init__(directed=True)
        if graphFile:
            self.graphFile = graphFile
            self._ReadFromFile(self.graphFile)

    def _ReadFromFile(self, graphFile:str):
        def ifFloat(x):
            try:
                return float(x)
            except ValueError:
                return x
        
        self.clear()
        with open(graphFile, 'r') as f:
            line = f.readline()
            while not line.startswith('# Nodes'):
                line = f.readline()
            nNodes = int(line.split(' ')[-1])
            attributes = f.readline().strip('\n').split(',')[1:] # First item is node name
            nodes = []
            for n in range(nNodes):
                node = f.readline().strip('\n').split(',')
                nodes.append((node[0], [ifFloat(node[i+1]) if attributes[i]!='position'
                                        else np.array(list(map(float, node[i+1].strip('[').strip(']').split()))) 
                                        for i in range(len(attributes))]))
            nodeAttributes = [n for n in attributes] # Copy
            line = f.readline()
            while not line.startswith('# Edges'):
                line = f.readline()
            nEdges = int(line.split(' ')[-1])
            attributes = f.readline().strip('\n').split(',')[2:] # First two items are node names
            edges = []
            for n in range(nEdges):
                edge = f.readline().split(',')
                edges.append((edge[0], edge[1], [ifFloat(edge[i+2]) for i in range(len(attributes))]))
            edgeAttributes = [n for n in attributes] # Copy
            
        # Convert to integers before adding to graph
        nodesToInt = {n[0]:i for i,n in enumerate(nodes)}
        nodes = [[nodesToInt[n], [get_prop_type(value, key)[1] for key, value in zip(nodeAttributes, d)]] for n,d in nodes]    
        edges = [(nodesToInt[n1], nodesToInt[n2], *[get_prop_type(value, key)[1] for key, value in zip(edgeAttributes, d)]) for n1,n2,d in edges]
        self.add_edge_list(edges,
                    eprops=[get_prop_type(value, key)[::-2] for key, value in zip(edgeAttributes, edges[0][2:])],        
                    )
        del edges, nodesToInt
        
        vprops = [self.new_vertex_property(get_prop_type(value,key)[0]) for key, value in zip(nodeAttributes, nodes[0][1])]
        for i, vprop in enumerate(vprops):
            self.vp[nodeAttributes[i]] = vprop
            if vprop.value_type() not in ['python::object','string', 'vector<double>', 'vector<long double>']:
                vprop.a = [d[i] for _,d in nodes]
            elif vprop.value_type() in ['string', 'python::object']:
                for n,d in nodes:
                    vprop[n] = d[i]
            else:
                vprop.set_2d_array(np.array([d[i] for _,d in nodes]).T, pos=np.array([n for n,_ in nodes]))
        del nodes
        return 
    
    def SplitVesselsToMeshSize(self, h:float, vessels:list):
        print("Not implemented yet.")
        pass

class ControlVolumes(object):
    '''
    Creates the control volumes within the Grid and stores the connections and their types
    between the grid and the control volumes (endothelial and (optional) vascular voxels)
    and between the vascular nodes and the control volumes.
    '''
    def __init__(self, grid:Grid, vessels:VascularGraph):
        # Compute distance to grid.origin along each axis
        d = vessels.new_vertex_property("bool")
        pos = vessels.vp['position']
        # #inFOV = (np.abs(np.array([pos[u] for u in vessels.iter_vertices()])-grid.origin)<=grid.l)
        # positions = vessels.vp['position'].get_2d_array([0,1,2]) - grid.origin.reshape((3,1))
        
        points = vessels.vp['position'].get_2d_array([0,1,2]) 
        d.a = np.logical_and((points<=grid.origin[:,np.newaxis]+grid.l[:,np.newaxis]), (points>=grid.origin[:,np.newaxis])).all(0)
        self.grid = grid
        self._vessels = vessels # Link to the original graph, if needed
        self.VAG = gt.GraphView(vessels, vfilt=d)

    def _RotationMatrix(self, n:np.ndarray):
        """Rotate a capsule onto the vessel axis n."""
        if (n==[0,0,1]).all():
            return np.eye(3)
        if (n==[0,0,-1]).all():
            R = np.array([[0,0,0],[0,0,0],[0,0,-1]])                                
        R = (n[2]-1)/(n[0]**2+n[1]**2) * np.array([[n[0]**2, n[0]*n[1], 0],
                                                    [n[0]*n[1], n[1]**2, 0],
                                                    [0, 0, n[0]**2+n[1]**2]])                      
        R+= np.array([[1, 0, -n[0]],
                        [0, 1, -n[1]],
                        [n[0], n[1], 1]])                              
        return R
    
    def _LabelRefCapsule(self, r:float, l:float, w:float,):
                        # nx:int=50, ny:int=50, nz:int=100):
        """
        Samples point around the refrence capsule 
        (a vertical cylinder of length l and radius r)
        and labels them according to the lumen radius and vessel wall thickness (w).
        """    
        # Creates a 3D grid, large enough for the capsule to fit in
        # pos    = np.meshgrid(np.linspace(-1.1*(r-w), 1.1*(r+w), endpoint=True, num=nx),
        #                     np.linspace(-1.1*(r-w), 1.1*(r+w), endpoint=True, num=ny),
        #                     np.linspace(0, l, endpoint=True, num=nz)) 
        width = r+w # Width of the cylinder
        pos = np.meshgrid(np.arange(-width, width, w/2),
                            np.arange(-width, width, w/2),
                            np.arange(-r, l+r, min(l/5, min(self.grid.h))))
        pos = np.vstack(list(map(lambda x:x.ravel(), pos))) # (3,nx*ny*nz) array of locations
        dists = np.linalg.norm(pos[:2], axis=0) # Distance from the cylinder axis
        labels = np.where(dists<r-w/2, 1, np.where(dists<r+w/2, 2, 0))
        # labels = np.where(dists<r+w, 2, 0)
        # labels[dists<r-w] = 1
        return pos, labels
    
    def _LabelAroundVessel(self, e:list, w:float):
        ''' 
        TODO: look for the closest node in the edge to assign the connectivity to. Should then return 4 sets (2 for each node). 
        ### Parameters:
        - e: list
            An edge.
        - w: float
            The endothelium thickness (in cm).
        '''

        p0, p1 = [self.VAG.vp['position'][n].a for n in e]
        r, l   = self.VAG.ep['radius'][e], self.VAG.ep['length'][e] 
        R = self._RotationMatrix((p1-p0)/l) # Rotation operator to the vessel axis

        pos, labels = self._LabelRefCapsule(r, l, w) # Return a grid with its labels on a vertical capsule
        m0 = pos[-1,:]<l/2 # The cells that should be attached to the proximal node's control volume
        pos = p0[:, np.newaxis] + R.T.dot(pos) # Transpose and translate the capsule to the real vessel's axis
        gridDomain = self.grid.PointToCell(pos) # Maps the points in the reference capsule to voxels in our grid
        
        mask = (gridDomain<self.grid.n[:,np.newaxis]).all(0) # Some points may be outside the grid after rotation, so we mask them
        mask = mask & (gridDomain>0).all(0)
        ijk = self.grid.ToFlatIndex(*gridDomain)[mask] # Maps the ijk indices to flat array indices
        try:
            endothelium0 = ijk[labels[mask]]
            endothelium1 = ijk[labels[mask]==2]
            vascular0    = ijk[labels[mask]==1]
            vascular1    = ijk[labels[mask]==1]
        except IndexError:
            print(ijk.shape, labels.shape, labels[mask].shape)
            raise IndexError("")
        vascular = set(vascular) # The vascular labels
        endothelium = set(endothelium) # The endothelium labels
        vascular = vascular-endothelium # Endothelium label is preffered over vascular  
        del pos, ijk, labels, gridDomain # Clean up          
        return endothelium, vascular
        
    def LabelMesh(self, w:float=1e-4)->None:
        '''
        TODO: see _LabelAroundVessel.
        Labels the mesh and saves the connectivity between the control volumes, the mesh and the vascular nodes. 
        ### Parameters
        - w: float
            The endothelium thickness (in cm).
        '''

        LabelVessel = self._LabelAroundVessel
        endo = self.VAG.new_vertex_property("python::object", val=set())
        vasc = self.VAG.new_vertex_property("python::object", val=set())

        for e in tqdm(self.VAG.iter_edges(), 
                        total=self.VAG.num_edges(), 
                        desc="Labelling...", unit="edge"): # Loop through the vessels in the FOV.
            # This should loop through the edges within the FOV, but the index of the vertices
            # is the same as in the original graph (therefore, the indices do not necesseraly start at 0).
            endo[e[0]], vasc[e[0]] = LabelVessel(e, w) # Save the sets of connections to the grid 
                                                                    # as vertex properties of the proximal node.
                                                                    # Should be connected to the closest node, but that's simpler.
            
        self.VAG.vp['endothelium'] = endo
        self.VAG.vp['vascular']    = vasc


    def bar(self): # To profile the time for labelling.
        profile.runctx('self.LabelMesh()', globals(), locals())

    def LabelsToGrid(self):
        # Write the labels to the grid for plotting.
        try:
            self.grid.labels = {i:1 for idx in self.VAG.vp['vascular'] for i in idx}
            self.grid.labels = {**self.grid.labels, **{i:2 for idx in self.VAG.vp['endothelium'] for i in idx}}        
        except KeyError:
            self.grid.labels = {} # No labels.
        
    def ToVTK(self, VTKFileName:str="gridLabels.vtk"): 
        self.LabelsToGrid()
        with open(VTKFileName, 'w') as f:
            f.write("# vtk DataFile Version 5.1\n")
            f.write("A mesh for the computation of oxygen perfusion.\n")
            #f.write("BINARY\n")
            f.write("ASCII\n")
            f.write("DATASET STRUCTURED_POINTS\n")
            f.write("DIMENSIONS {:d} {:d} {:d}\n".format(*(self.grid.n+1)))
            f.write("ORIGIN {:f} {:f} {:f}\n".format(*self.grid.origin))
            f.write("SPACING {:f} {:f} {:f}\n".format(*self.grid.h))
        
            # Writing the data
            f.write(f"CELL_DATA {self.grid.size}\n")
            f.write("SCALARS labels int 1\n")
            f.write("LOOKUP_TABLE default\n")

            a = np.array(
                [
                    self.grid.labels.get(i,0) for i in 
                        tqdm(
                            # Change the order to x change the fastest and z the slowest for paraview.
                                np.arange(self.grid.size).reshape(self.grid.shape).ravel('F'), 
                            unit='labels',
                            desc=f"Writing labels to {VTKFileName}",
                            total=self.grid.size)
                ],
                dtype=np.int8
                )
            a.tofile(f, sep='\n', format='%d')
            del a

            # f.write(('\n'.join(i.__str__() for i in 
            #                    tqdm(self.grid, desc=f"Writing labels to {VTKFileName}", 
            #                         total=self.grid.size, unit=' Labels'))))   
        return          

    def _PlotPlexus(self, plexus:int, output=None):
        _G = gt.GraphView(self.VAG, vfilt=self.VAG.vp['plexus'].fa==plexus)
        if _G.num_vertices()==0:
                print(f"No vertices in the " + ("SVP" if plexus==0 else
                                            "ICP" if plexus==1 else
                                            "DCP") + " layer. Not plotting.")
                return
        graph_draw(_G, pos=self.VAG.vp['position'], bg_color='white', output=output)
        del _G
    def PlotSVP(self, output=None):
        self._PlotPlexus(plexus=0, output=output)             
    def PlotICP(self, output=None):
        self._PlotPlexus(plexus=1, output=output)
    def PlotDCP(self, output=None):
        self._PlotPlexus(plexus=2, output=output)           


In [36]:
def LabelACapsule(r:float, l:float, w:float, nx:int=20, ny:int=20, nz:int=20):
    """
    Samples point around the refrence capsule 
    (a vertical cylinder of length l and radius r)
    and labels them according to the lumen radius and vessel wall thickness (w).
    """    
    # pos    = np.meshgrid(np.linspace(-1.5*r, 1.5*r, endpoint=True, num=nx),
    #                      np.linspace(-1.5*r, 1.5*r, endpoint=True, num=ny),
    #                      np.linspace(0, l, endpoint=True, num=nz))
    width = r+1.2*w # Width of the cylinder
    pos = np.meshgrid(np.arange(-width, width, w*2),
                        np.arange(-width, width, w*2),
                        np.arange(0, 1*l, w))
    nx, ny, nz = pos[0].shape
    pos = np.vstack(list(map(lambda x:x.ravel(), pos))) # (3,nx*ny*nz) array of locations
    dists = np.linalg.norm(pos[:2], axis=0) # Distance from the cylinder axis
    labels = np.where(dists<=r-w, 1, np.where(dists<r+w, 2, 0)) # Vascular voxels first, then endothelial
    return pos.reshape((3,nx,ny,nz)), labels.reshape((nx,ny,nz))     
def AddIntermediateVertices(graph:VascularGraph, edge:list[int], num_intermediate:int):
    if num_intermediate<1:
        return
    source, target = edge
    originalEdgeProp = {key: graph.ep[key][edge] for key in graph.ep if key!='length'}
    originalNodeProp = {key: graph.vp[key][source] for key in graph.vp if key!='position'}
    p0, p1 = graph.vp['position'][source].a, graph.vp['position'][target].a
    l = np.linalg.norm(p0-p1)/(num_intermediate+1)
    n = (p1-p0)/np.linalg.norm(p0-p1)
    graph.remove_edge(edge)
    newNodes = graph.add_vertex(num_intermediate)
    for i, newNode in enumerate(newNodes if num_intermediate>1 else [newNodes]):   
        for key, value in originalNodeProp.items():
            graph.vp[key][newNode] = value  
        graph.vp['position'][newNode] = p0 + n*l*(i+1)     
        newEdge = graph.add_edge(source, newNode)
        for key, value in originalEdgeProp.items():
            graph.ep[key][newEdge] = value 
        graph.ep['length'][newEdge] = l
        source = newNode
    newEdge = graph.add_edge(newNode, target)
    for key, value in originalEdgeProp.items():
        graph.ep[key][newEdge] = value 
    graph.ep['length'][newEdge] = l

def SplitVessels(graph:VascularGraph, h:list[float]):
    edges = list(graph.iter_edges())
    positions = graph.vp['position']
    num_intermediates = abs(np.array([[positions[u].a-positions[v].a] for u,v in edges]))/np.asarray(h)
    num_intermediates = np.squeeze(num_intermediates, 1).max(-1).astype(int)    
    for edge, num_intermediate in tqdm(zip(edges, num_intermediates), desc='Refining vascular mesh', total=len(edges), unit='edges'):
        if num_intermediate<1:
            continue
        AddIntermediateVertices(graph, edge, num_intermediate+1)
    return

### Test the labelling through reference capsule and rotation

In [37]:
# Randomly oriented vessel
# p0, p1 = np.random.random(3), np.random.random(3)
p0 = np.array([0.01,0.01,0.01])
p1 = np.array([0.01,0.08,0.01])
n = (p1-p0)/np.linalg.norm(p0-p1, 2)
r = 0.0025
l = np.linalg.norm(p0-p1, 2)
k = 1
VAG = VascularGraph()
VAG.add_edge_list([(i,i+1,l/k, r) for i in range(k)], eprops=[('length', 'float'), ('radius','float')])
pos = VAG.new_vertex_property("vector<double>")
pos.set_2d_array(np.array([p0+t*(p1-p0) for t in np.linspace(0,1,endpoint=True, num=k+1)]).T)
VAG.vp['position'] = pos
VAG.vp['plexus'] = VAG.new_vertex_property("int", val=0)
# gt.draw.graph_draw(VAG, pos=VAG.vp['position'])
CV = ControlVolumes(Grid((0,0,0), (0.1,.1,.1), (300,300,200)), VAG)
SplitVessels(CV.VAG, CV.grid.h)
CV.LabelMesh(1e-4)
CV.ToVTK("LabelGridOneVessel.vtk")

Refining vascular mesh:   0%|          | 0/1 [00:00<?, ?edges/s]

Labelling...:   0%|          | 0/211 [00:00<?, ?edge/s]

(3, 54080) (3, 54080) (54080,)


ValueError: 

In [17]:
VAG = VascularGraph("/media/Storage3.6TB/DataForIOVSPaper/200Sims/OCTAMetrics/sim_100_AV.graph")
# randomNum = np.random.random(VAG.num_vertices())
# VAG.remove_vertex([v for v,p in zip(VAG.iter_vertices(), randomNum) if p>0.2])

#VAG.remove_vertex((u for u,p in zip(VAG.iter_vertices(), VAG.vp['plexus']) if p!=0))
lx, ly, lz = .3,.3,.02
# #VAG.ep.radius.a /= 1e4
CV = ControlVolumes(Grid((-lx/2, -ly/2, -lz), (lx,ly,lz*1.2), (700,700,300)), VAG)

#gt.draw.graph_draw(CV.VAG, pos=CV.VAG.vp['position'], bg_color='white')
SplitVessels(CV.VAG, CV.grid.h)
# z = np.zeros((3,5))
# z[-1,:] += np.linspace(0,1,5) 
# newPoints = []
# oldPoints = []
# for e in CV.VAG.iter_edges():
#     p0,p1 = [CV.VAG.vp['position'][n].a for n in e]    
#     oldPoints.append([p0,p1])
#     l = np.linalg.norm(p1-p0)
#     n = (p1-p0)/l
#     R = CV._RotationMatrix(n)
#     points = (p0[:, np.newaxis] + l*R.T.dot(z)).T
#     points = CV.grid.CellCenter(CV.grid.PointToCell(points.T)).T
#     for i in range(points.shape[0]-1):
#         p0,p1 = points[i], points[i+1]# [CV.grid.CellCenter(CV.grid.PointToCell(points[i+j])) for j in range(2)]
#         newPoints.append([p0,p1])
# newPoints = np.array(newPoints)[:,:,:2]
# oldPoints = np.array(oldPoints)[:,:,:2]
# print(newPoints.shape, oldPoints.shape)
# fig, (ax1,ax2) = plt.subplots(1,2, sharex=True, sharey=True)
# line1 = LineCollection(newPoints, color='k')
# line2 = LineCollection(oldPoints, color='k')
# ax1.add_collection(line1)
# ax2.add_collection(line2)

# minx, miny = np.array([CV.VAG.vp['position'][n][:2] for n in CV.VAG.iter_vertices()]).min(0)
# maxx, maxy = np.array([CV.VAG.vp['position'][n][:2] for n in CV.VAG.iter_vertices()]).max(0)
# for ax in (ax1,ax2):
#     ax.set_xlim(minx,maxx)
#     ax.set_ylim(miny,maxy)

# ax1.plot(*newPoints[:2], c='k')
# ax2.plot(*oldPoints[:2], c='k')
plt.show()
# gt.draw.graph_draw(CV.VAG, pos=CV.VAG.vp['position'], bg_color='white')
CV.LabelMesh(1e-4)
# #CV.bar()
#CV.ToVTK()

Refining vascular mesh:   0%|          | 0/38577 [00:00<?, ?edges/s]

Labelling...:   0%|          | 0/365117 [00:00<?, ?edge/s]

In [18]:
# fig, ax = plt.subplots(figsize=(6,6))
# ax.plot(*oldPoints[:2], c='k')

# CV = ControlVolumes(Grid((-lx/2, -ly/2, -lz), (lx,ly,lz*1.2), (200,200,200)), VAG)
# CV.LabelMesh()
# labels = np.array([[[CV.grid.labels.get(CV.grid.ToFlatIndex(i,j,k), 0) for k in range(CV.grid.n[-1])]
#                    for j in range(CV.grid.n[1])] for i in range(CV.grid.n[0])])
# X,Y = np.meshgrid(np.linspace(oldPoints[0].min(), oldPoints[0].max(), labels.shape[0]),
#                   np.linspace(oldPoints[1].min(), oldPoints[1].max(), labels.shape[1]))
# labels = ma.masked_where(labels==0, labels)
# cb = ax.pcolormesh(X.T, Y.T, labels.max(-1), alpha=1)
# plt.colorbar(cb)
# plt.show()

# Construction of matrices
## O_2 convection in the vascular domain
### Surface area scaling operator

In [25]:
import scipy.sparse as sp
Np, Nf, Nt, Nc = CV._vessels.num_vertices(), CV._vessels.num_edges(), CV.grid.size, CV.VAG.num_vertices()
C2v, C2t, N = sp.dok_array((Nc,Np)), sp.dok_array((Nc,Nt)), np.zeros((Nc,))
for v in CV.VAG.iter_vertices():
    endo = CV.VAG.vp['endothelium'][v]
    assert len(endo)>0, f"No endothelium label for this vertex: {v}"

AssertionError: No endothelium label for this vertex: 1